In [4]:
import pandas as pd
import numpy as np
# This will be our data cleaners and fixers
from newsapi import NewsApiClient
# This is how we'll be getting news data to discover sentiment
import nltk
# Our nlp pipeline, for any nlp cleaning tasks
from textblob import TextBlob
# One method on gathering sentiment (since this will be unsupervised learning)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Second method of gathering sentimnet
from iex import Stock
# THis is how we'll be getting stock data, and find correlation between movement and setiment generated
import plotly_express as px
#Nice graphs
import seaborn as sns
# Nicer graphs than matplotlib
import plotly.graph_objs as go
# For More functionality with plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
from spacy.tokens import Doc
import spacy

plotly.tools.set_credentials_file(username='EddyP', api_key='VrIJ8CKqnghjth6SCJoS')

newsapi = NewsApiClient(api_key='22901f790d59422e817ef038a62c6c38')

stocks_df = pd.read_csv('data/constituents.csv')
#stocks_df.head()
sample = stocks_df[stocks_df['Sector'] == 'Industrials']


In [5]:
sample.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
6,AYI,Acuity Brands Inc,Industrials
17,ALK,Alaska Air Group Inc,Industrials
22,ALLE,Allegion,Industrials


In [6]:
def get_data(data):
    my_df = {}
    mashed_df = [] # We will concatinate all dataframes to measure overall sentiment
    stock_df = {}
    news_records = {}
    for name,ticker in zip(data['Name'], data['Symbol']):
        data = newsapi.get_everything(q = name)
        news_records[ticker] = data
        df = pd.DataFrame(data['articles'])
        df['publishedAt'] = [i.replace('T',' ') for i in df['publishedAt']]
        df['publishedAt'] = [i.replace('Z',' ') for i in df['publishedAt']]
        df['publishedAt'] = pd.to_datetime(df['publishedAt'])
        #df = df.set_index['publishedAt']
        #df = df.sort_index()
        mashed_df.append(df)
        my_df[ticker] = df
        stock_df[ticker] = Stock(ticker).chart_table(range='3M')
    return my_df, mashed_df, stock_df

In [7]:
my_df , mashed_df, stock_df = get_data(sample)

https://api.iextrading.com/1.0/stock/MMM/chart/3M
https://api.iextrading.com/1.0/stock/AOS/chart/3M
https://api.iextrading.com/1.0/stock/AYI/chart/3M
https://api.iextrading.com/1.0/stock/ALK/chart/3M
https://api.iextrading.com/1.0/stock/ALLE/chart/3M
https://api.iextrading.com/1.0/stock/AAL/chart/3M
https://api.iextrading.com/1.0/stock/AME/chart/3M
https://api.iextrading.com/1.0/stock/ARNC/chart/3M
https://api.iextrading.com/1.0/stock/BA/chart/3M
https://api.iextrading.com/1.0/stock/CHRW/chart/3M
https://api.iextrading.com/1.0/stock/CAT/chart/3M
https://api.iextrading.com/1.0/stock/CTAS/chart/3M
https://api.iextrading.com/1.0/stock/CSX/chart/3M
https://api.iextrading.com/1.0/stock/CMI/chart/3M
https://api.iextrading.com/1.0/stock/DE/chart/3M
https://api.iextrading.com/1.0/stock/DAL/chart/3M
https://api.iextrading.com/1.0/stock/DOV/chart/3M
https://api.iextrading.com/1.0/stock/ETN/chart/3M
https://api.iextrading.com/1.0/stock/EMR/chart/3M
https://api.iextrading.com/1.0/stock/EFX/chart/3

In [8]:
def adding_tickers(stock_data,ticker_symbols):
    for i in ticker_symbols:
        stock_data[i]['ticker'] = i
    return stock_data

In [9]:
def sentimentizer_textblob(news_data,ticker): # News data is going to be the ticker name + News according to it
    # The cleaning and processing part!
    news_df = news_data[ticker]
    try:
        news_df = news_df.set_index('publishedAt')
    except:
        pass
    news_df = news_df.sort_index()
    news_df['content'] = news_df['content'].fillna('None')
    news_df['description'] = news_df['description'].fillna('None')
    news_df['title'] = news_df['title'].fillna('None')
    # Textblober, to create objects that can then be interacted with.
    news_df['content_wiki'] = [TextBlob(i) for i in news_df['content']]
    news_df['description_wiki'] = [TextBlob(i) for i in news_df['description']]
    news_df['title_wiki']= [TextBlob(i) for i in news_df['title']]
    news_df['content_polarity'] = [i.polarity for i in news_df['content_wiki']]
    news_df['description_polarity'] = [i.polarity for i in news_df['description_wiki']]
    news_df['title_polarity'] = [i.polarity for i in news_df['title_wiki']]
    news_df['content_subjectivity'] = [i.subjectivity for i in news_df['content_wiki']]
    news_df['description_subjectivity'] = [i.subjectivity for i in news_df['description_wiki']]
    news_df['title_subjectivity'] = [i.subjectivity for i in news_df['title_wiki']]
    news_df['Date'] = news_df.index.astype(str)
    news_df['Date'] = [i[0:10] for i in news_df.Date]
    news_df['Time'] = [i[12:19] for i in news_df.Date]

    textblob_sentiment_df = news_df[['content_polarity','description_polarity','Date']]
    textblob_sentiment_df = textblob_sentiment_df.groupby('Date').agg('mean')
    textblob_sentiment_df['avg_sentiment'] = [np.mean([i,j]) for i,j in zip(textblob_sentiment_df['content_polarity'],
                                                                       textblob_sentiment_df['description_polarity'])]
    textblob_sentiment_df['sum_sent'] = [np.sum([i,j]) for i,j in zip(textblob_sentiment_df['content_polarity'],
                                                                       textblob_sentiment_df['description_polarity'])]
    textblob_sentiment_df
    return news_df, textblob_sentiment_df

In [10]:
sentiment_analyzer = SentimentIntensityAnalyzer()
nlp = spacy.load('en')
def polarity_scores(doc):
    return sentiment_analyzer.polarity_scores(doc.text)

In [30]:
def sentimetizer_vader(news_df,ticker_symbol):
    # Creating our vader objects so we can interact with their properties
    #if ticker_symbol
    news_df = news_df[ticker_symbol]
    news_df['content_vader'] = [nlp(i) for i in news_df['content']]
    news_df['description_vader'] = [nlp(i) for i in news_df['description']]
    news_df['title_vader']= [nlp(i) for i in news_df['title']]
    # Now we have what we're looking for!
    news_df['content_vader_output'] = [polarity_scores(i) for i in news_df['content_vader']]
    news_df['description_vader_output'] = [polarity_scores(i) for i in news_df['description_vader']]
    news_df['title_vader_output'] = [polarity_scores(i) for i in news_df['title_vader']]
    #small problem. They create a dictionary that just holds
    # BUT we don't care about this, we will send our data and concatinate it with the correspoding stock dataframe!
    # Let's copy our news_df
#    def sentiment_data_frame(news_df[['content_vader_output','description_vader_output','title_vader_output']]):
#        cvo_df = pd.DataFrame(news_df['content_vader_output'])
#        dvo_df = pd.DataFrame(news_df['description_vader_output'])
#        tvo_df = pd.DataFrame(news_df['title_vader_output'])

    cvo_df = pd.DataFrame(news_df['content_vader_output'])
    cvo_df = cvo_df.reset_index()
    data = pd.DataFrame(cvo_df['content_vader_output'].tolist())
    index_series = pd.Series(cvo_df.index)
    d1 = data.reset_index
    d2 = index_series
    time_df = cvo_df['publishedAt']
    time_df = time_df.reset_index()
    data_2 = data.reset_index()
    fixed_df = data_2.merge(time_df)
    fdf = fixed_df.set_index('publishedAt')
    content_sentiment = fixed_df.set_index('publishedAt')
    content_sentiment.head()
    content_sentiment['Full_Time'] = content_sentiment.index.astype(str)
    #content_sentiment['date'] = [str(i[0:10]) for i in content_sentiment['pubAt']]
    #content_sentiment['pubAt'] = content_sentiment['pubAt']
    content_sentiment['Date'] = [i[0:10] for i in content_sentiment['Full_Time']]
    content_sentiment['day'] = [i[0:10] for i in content_sentiment['Date']]
    content_sentiment['time'] = [i[12:19] for i in content_sentiment['Full_Time']]
    content_sentiment['time'] = pd.to_datetime(content_sentiment['time'],format='%H:%M:%S').dt.time
    mean_sentiment = content_sentiment.groupby('Date').agg('mean')
    to_work = mean_sentiment.reset_index()

    return news_df , content_sentiment, mean_sentiment

In [34]:
def sentiment_gathering(textblob_sentiment, vader_sentiment, stock_df,ticker):
    df1 = stock_df[ticker]
    df2 = textblob_sentiment
    d3 = vader_sentiment
    df1['Date'] = df1['date']
    df1 = df1.set_index('Date')
    sentiment_df = df2.join(d3)
    semi_final_df = df1.join(sentiment_df)
    df6 = semi_final_df
    final_df = df6.fillna(method='ffill',inplace = True)
    return semi_final_df.fillna(0)

In [13]:
testing = stock_df
stock_df= adding_tickers(stock_data=testing,ticker_symbols=sample['Symbol'])

In [28]:
my_df['MMM']

,author,content,description,publishedAt,source,title,url,urlToImage,content_vader,description_vader,title_vader,content_vader_output,description_vader_output,title_vader_output
0,Investing.com,Investing.com – Wall Street was mixed on Thurs...,"Stocks - Dow Falls After 3M Shock; Microsoft, ...",2019-04-25 13:46:00,"{'id': None, 'name': 'Investing.com'}","Stocks - Dow Falls After 3M Shock; Microsoft, ...",https://www.investing.com/news/stock-market-ne...,https://i-invdn-com.akamaized.net/news/LYNXMPE...,"(Investing.com, –, Wall, Street, was, mixed, o...","(Stocks, -, Dow, Falls, After, 3, M, Shock, ;,...","(Stocks, -, Dow, Falls, After, 3, M, Shock, ;,...","{'neg': 0.072, 'neu': 0.854, 'pos': 0.074, 'co...","{'neg': 0.136, 'neu': 0.864, 'pos': 0.0, 'comp...","{'neg': 0.206, 'neu': 0.794, 'pos': 0.0, 'comp..."
1,Victoria Song,"Everyone has an opinion about fonts, but Helve...","Everyone has an opinion about fonts, but Helve...",2019-04-17 20:00:00,"{'id': None, 'name': 'Gizmodo.com'}","After 36 Years, Helvetica Gets a Much-Needed F...",https://gizmodo.com/after-36-years-helvetica-g...,https://i.kinja-img.com/gawker-media/image/upl...,"(Everyone, has, an, opinion, about, fonts, ,, ...","(Everyone, has, an, opinion, about, fonts, ,, ...","(After, 36, Years, ,, Helvetica, Gets, a, Much...","{'neg': 0.0, 'neu': 0.927, 'pos': 0.073, 'comp...","{'neg': 0.0, 'neu': 0.923, 'pos': 0.077, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,Neha Mistry,"At some point in life, all of us have used, lo...",3M needs a Senior Interaction Designer At some...,2019-04-15 10:00:51,"{'id': None, 'name': 'Yankodesign.com'}",3M needs a Senior Interaction Designer,https://www.yankodesign.com/2019/04/15/3m-need...,None,"(At, some, point, in, life, ,, all, of, us, ha...","(3, M, needs, a, Senior, Interaction, Designer...","(3, M, needs, a, Senior, Interaction, Designer)","{'neg': 0.0, 'neu': 0.85, 'pos': 0.15, 'compou...","{'neg': 0.0, 'neu': 0.877, 'pos': 0.123, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,Tomi Kilgore,Shares of 3M Co. plummeted toward its biggest ...,Shares of 3M Co. plunge toward biggest one-day...,2019-04-25 17:11:15,"{'id': None, 'name': 'Marketwatch.com'}",3M’s shares on verge of worst day since 1987 s...,https://www.marketwatch.com/story/3ms-shares-o...,http://s.marketwatch.com/public/resources/MWim...,"(Shares, of, 3, M, Co., plummeted, toward, its...","(Shares, of, 3, M, Co., plunge, toward, bigges...","(3, M, ’s, shares, on, verge, of, worst, day, ...","{'neg': 0.12, 'neu': 0.771, 'pos': 0.109, 'com...","{'neg': 0.119, 'neu': 0.704, 'pos': 0.177, 'co...","{'neg': 0.4, 'neu': 0.471, 'pos': 0.129, 'comp..."
4,Mark DeCambre,U.S. stock-index futures on Thursday morning w...,U.S. stock-index futures on Thursday morning t...,2019-04-25 12:01:38,"{'id': None, 'name': 'Marketwatch.com'}",Market Snapshot: Dow futures under pressure as...,https://www.marketwatch.com/story/dow-futures-...,http://s.marketwatch.com/public/resources/MWim...,"(U.S., stock, -, index, futures, on, Thursday,...","(U.S., stock, -, index, futures, on, Thursday,...","(Market, Snapshot, :, Dow, futures, under, pre...","{'neg': 0.111, 'neu': 0.784, 'pos': 0.105, 'co...","{'neg': 0.14, 'neu': 0.727, 'pos': 0.132, 'com...","{'neg': 0.193, 'neu': 0.807, 'pos': 0.0, 'comp..."
5,newsfeedback@fool.com (Neha Chamaria),When 3M(NYSE:MMM)downgraded its sales guidance...,From macro headwinds to operational inefficien...,2019-04-26 14:27:00,"{'id': None, 'name': 'Fool.com'}",3M's Dud Quarter Reveals What's Hurting the Co...,https://www.fool.com/investing/2019/04/26/3ms-...,https://g.foolcdn.com/editorial/images/520506/...,"(When, 3M(NYSE, :, MMM)downgraded, its, sales,...","(From, macro, headwinds, to, operational, inef...","(3, M, 's, Dud, Quarter, Reveals, What, 's, Hu...","{'neg': 0.087, 'neu': 0.782, 'pos': 0.131, 'co...","{'neg': 0.074, 'neu': 0.926, 'pos': 0.0, 'comp...","{'neg': 0.439, 'neu': 0.561, 'pos': 0.0, 'comp..."
6,newsfeedback@fool.c

In [32]:
news_df = {}
my_textblob_sentiment_df = {}
mean_sentiment={}
content_sentiment = {}
my_sentiment_data = {}
for i in sample['Symbol']:
    news_df[i], my_textblob_sentiment_df[i] = sentimentizer_textblob(my_df,i)
    my_news_df[i] , content_sentiment[i], mean_sentiment[i]= sentimetizer_vader(news_df,i)

In [46]:
sentiment_data = {}
for i in sample['Symbol']:
    sentiment_data[i] = sentiment_gathering(my_textblob_sentiment_df[i], mean_sentiment[i], stock_df,i)

In [47]:
big_df = [sentiment_data[ticker] for ticker in sample['Symbol']]

In [48]:
big_data = pd.concat(big_df, axis= 0)
from sklearn import preprocessing
# compound1 = go.Scatter(x =df4['Time'], y = df4['compound'])
# pchanges = go.Scatter(x = df4['Time'],y=df4['Close_Changes'])
data = big_data
data["Time"] = data.index
X = np.array(data['compound'].values)
Y=  np.array(data['change'].values)
Z = np.array(data['avg_sentiment'].values)
K = np.array(data['sum_sent'].values)
std_x = preprocessing.scale(X)
std_y = preprocessing.scale(Y)
std_z = preprocessing.scale(Z)
std_k = preprocessing.scale(K)
data['Scale_compound'] = std_x
data['Scale_pctchng'] = std_y
data['Scale_avg']=std_z
data['Scale_sum'] = std_k

In [49]:
big_data.to_csv('data/results.csv')

In [52]:
big_data.shape

(4156, 27)

In [56]:
tickers = set
tickers = set(big_data['ticker'])
tickers

{'AAL',
 'ALK',
 'ALLE',
 'AME',
 'AOS',
 'ARNC',
 'AYI',
 'BA',
 'CAT',
 'CHRW',
 'CMI',
 'COL',
 'CSX',
 'CTAS',
 'DAL',
 'DE',
 'DOV',
 'EFX',
 'EMR',
 'ETN',
 'EXPD',
 'FAST',
 'FBHS',
 'FDX',
 'FLR',
 'FLS',
 'FTV',
 'GD',
 'GE',
 'GWW',
 'HII',
 'HON',
 'INFO',
 'IR',
 'ITW',
 'JBHT',
 'JCI',
 'JEC',
 'KSU',
 'LLL',
 'LMT',
 'LUV',
 'MAS',
 'MMM',
 'NLSN',
 'NOC',
 'NSC',
 'PCAR',
 'PH',
 'PNR',
 'PWR',
 'RHI',
 'ROK',
 'ROP',
 'RSG',
 'RTN',
 'SRCL',
 'TDG',
 'TXT',
 'UAL',
 'UNP',
 'UPS',
 'URI',
 'UTX',
 'VRSK',
 'WM',
 'XYL'}

In [60]:
from plotly import tools
import plotly.plotly as py
data = big_data[big_data['ticker'] == 'UPS']
fig = tools.make_subplots(rows=2,cols=1, shared_xaxes=True)

for col in data[['open','close']]:
    fig.append_trace({'x':data.Time,'y':data[col],'type':'scatter', 'name':col},1,1)
for col,title in zip(data[['Scale_compound','Scale_pctchng','Scale_avg']],['Compound Sentiment', 'Changes','Avg Sentiment']):
    fig.append_trace({'x':data.Time,'y':data[col],'type':'scatter', 'name':title},2,1)

py.plot(fig)


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



'https://plot.ly/~Eddyp/71'